In [ ]:
# -----------------------------
# Paso 0: Importar librerías
# -----------------------------
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import plotly.graph_objects as go
from sklearn.decomposition import PCA

In [ ]:
# -----------------------------
# Paso 1: Cargar dataset de entrenamiento
# -----------------------------

df = pd.read_excel("dataset_entrenamiento_1053.ods", engine="odf")

# Normalizar la columna 'tipo_ambiguedad' (minúsculas, sin espacios)
df["tipo_ambiguedad"] = df["tipo_ambiguedad"].astype(str).str.lower().str.strip()

# Tipos que se consideran ambiguos
tipos_ambiguos = ["semántica", "pragmática", "sintáctica", "léxica",
                  "semantica", "pragmatica", "sintantica", "lexica"]

# Crear columna 'ambiguo' (1 si es ambiguo, 0 si no)
df["ambiguo"] = df["tipo_ambiguedad"].apply(lambda x: 1 if x in tipos_ambiguos else 0)

# Crear la estructura corpus_requerimientos
corpus_requerimientos = [
    {"requerimiento": req, "ambiguo": amb}
    for req, amb in zip(df["requerimiento"], df["ambiguo"])
]

# Crear listas separadas
requerimientos = [r["requerimiento"] for r in corpus_requerimientos]
labels = [r["ambiguo"] for r in corpus_requerimientos]

In [ ]:
# -----------------------------
# Paso 3: Entrenar el modelo y graficamos los resultados
# -----------------------------

# Fijar semilla global
SEED = 42

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# Generar embeddings
embedder = SentenceTransformer('intfloat/multilingual-e5-large')
X = embedder.encode(requerimientos)
Y = np.array(labels)

# Dividir datos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(
    X, Y, test_size=0.2, random_state=SEED, stratify=Y
)

# Crear Red neuronal profunda para Clasificación Binaria

model = Sequential([
    Dense(256, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),

    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(64, activation='relu'),
    Dropout(0.2),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(16, activation='relu'),
    Dropout(0.1),

    Dense(1, activation='sigmoid')  # Salida binaria con sigmoid
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Definir EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Entrenamiento con EarlyStopping
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=110,
    batch_size=2,
    verbose=1,
    callbacks=[early_stop]
)

# Visualizar los resulatdos de Loss y Accuracy

# Loss
plt.figure(figsize=(10,4))
plt.plot(history.history['loss'], label='Loss de Entrenamiento', color='red')
plt.plot(history.history['val_loss'], label='Loss de Validación', color='salmon', linestyle='--')
plt.xlabel('Época')
plt.ylabel('Loss (Crossentropy Binaria)')
plt.title('Evolución del Loss durante el entrenamiento')
plt.legend()
plt.show()

# Accuracy
plt.figure(figsize=(10,4))
plt.plot(history.history['accuracy'], label='Accuracy de Entrenamiento', color='blue')
plt.plot(history.history['val_accuracy'], label='Accuracy de Validación', color='skyblue', linestyle='--')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.title('Evolución del Accuracy durante el entrenamiento')
plt.legend()
plt.show()

# -Reducir embeddings a 3D con PCA
pca = PCA(n_components=3)
X_3d = pca.fit_transform(X)

# Crear malla 3D para frontera de decisión
h = 0.1
x_min, x_max = X_3d[:, 0].min() - 0.5, X_3d[:, 0].max() + 0.5
y_min, y_max = X_3d[:, 1].min() - 0.5, X_3d[:, 1].max() + 0.5
z_min, z_max = X_3d[:, 2].min() - 0.5, X_3d[:, 2].max() + 0.5

xx, yy, zz = np.meshgrid(
    np.arange(x_min, x_max, h),
    np.arange(y_min, y_max, h),
    np.arange(z_min, z_max, h)
)

grid_points = np.c_[xx.ravel(), yy.ravel(), zz.ravel()]
grid_original_dim = pca.inverse_transform(grid_points)
Z = model.predict(grid_original_dim, verbose=0).flatten()


# Graficar con Plotly
num_samples = 2000
total_population = grid_points.shape[0]
num_samples = min(num_samples, total_population)

indices = np.random.choice(total_population, size=num_samples, replace=False)

sample_points = grid_points[indices]
sample_Z = Z[indices]

# Graficar
fig = go.Figure()

# Puntos de la malla (frontera de decisión)
fig.add_trace(go.Scatter3d(
    x=sample_points[:,0],
    y=sample_points[:,1],
    z=sample_points[:,2],
    mode='markers',
    marker=dict(
        size=3,
        color=sample_Z,
        colorscale='RdBu',
        opacity=0.3,
        colorbar=dict(title='Probabilidad (Ambiguo)')
    ),
    name='Frontera de Decisión'
))

# Puntos reales
fig.add_trace(go.Scatter3d(
    x=X_3d[:,0],
    y=X_3d[:,1],
    z=X_3d[:,2],
    mode='markers+text',
    marker=dict(
        size=8,
        color=Y,
        colorscale='RdBu',
        opacity=1
    ),
    text=[str(label) for label in labels],
    textposition="top center",
    name='Datos reales (0: No Ambiguo, 1: Ambiguo)'
))

fig.update_layout(
    scene=dict(
        xaxis_title='Componente 1',
        yaxis_title='Componente 2',
        zaxis_title='Componente 3'
    ),
    title="Embeddings 3D y Frontera de Decisión (Clasificación Binaria)",
    width=900,
    height=700
)

fig.show()

In [ ]:
# -----------------------------
# Paso 4: Cargar dataset de prueba
# -----------------------------

# Cargar archivo y preparar datos
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/dataset_prueba_263.ods", engine="odf")

# Normalizar la columna 'tipo_ambiguedad' (minúsculas, sin espacios)
df["tipo_ambiguedad"] = df["tipo_ambiguedad"].astype(str).str.lower().str.strip()

# Tipos que se consideran ambiguos
tipos_ambiguos = ["semántica", "pragmática", "sintáctica", "léxica",
                  "semantica", "pragmatica", "sintantica", "lexica"]

# Crear columna binaria 'ambiguo' (1 si es ambiguo, 0 si no)
df["ambiguo"] = df["tipo_ambiguedad"].apply(lambda x: 1 if x in tipos_ambiguos else 0)

# Crear la estructura de prueba
requerimientos_prueba = [
    {"requerimiento": req, "ambiguo": amb, "tipo": tipo}
    for req, amb, tipo in zip(df["requerimiento"], df["ambiguo"], df["tipo_ambiguedad"])
]

In [ ]:

# -----------------------------
# Paso 5: Probar el modelo
# -----------------------------

total_aciertos = 0
total_fallos = 0

# Contar cuántos datos hay por tipo
total_por_tipo = df["tipo_ambiguedad"].value_counts().to_dict()

# Inicializar diccionario de aciertos por tipo
aciertos_por_tipo = {tipo: 0 for tipo in df["tipo_ambiguedad"].unique()}

# Inicializar contadores de VN, VP, FN, FP
VN = 0
VP = 0
FN = 0
FP = 0

for req in requerimientos_prueba:
    texto = req["requerimiento"]
    tipo = req["tipo"]
    real_label = req["ambiguo"]

    # Generar embedding
    vec = embedder.encode([texto])

    # Predicción
    pred_prob = model.predict(vec)[0][0]
    pred_label = 1 if pred_prob > 0.5 else 0

    # Comparar con el valor real
    if pred_label == real_label:
        total_aciertos += 1
        aciertos_por_tipo[tipo] = aciertos_por_tipo.get(tipo, 0) + 1
        if pred_label == 1:
            VP += 1
        else:
            VN += 1
    else:
        total_fallos += 1
        if pred_label == 1:
            FP += 1
        else:
            FN += 1

    # Mostrar detalle por requerimiento
    resultado = "Ambiguo" if pred_label == 1 else "No ambiguo"
    real = "Ambiguo" if real_label == 1 else "No ambiguo"
    print(f"Requerimiento: {texto}")
    print(f" → Predicción: {resultado} (prob={pred_prob:.2f}) | Real: {real} | Tipo: {tipo}\n")

# Resultados totales
total = total_aciertos + total_fallos
porc_aciertos = (total_aciertos / total) * 100
porc_fallos = (total_fallos / total) * 100

print("\n===============================")
print(f"✅ Total de aciertos: {total_aciertos} ({porc_aciertos:.2f}%)")
print(f"❌ Total de fallos: {total_fallos} ({porc_fallos:.2f}%)")
print("===============================\n")

# Mostrar métricas de VN, VP, FN, FP
print("📊 Matriz de confusión:")
print(f" - VP (Verdaderos positivos): {VP}")
print(f" - VN (Verdaderos negativos): {VN}")
print(f" - FP (Falsos positivos): {FP}")
print(f" - FN (Falsos negativos): {FN}\n")

# Aciertos por tipo de ambigüedad
print("📊 Aciertos por tipo de ambigüedad:")
for tipo in aciertos_por_tipo.keys():
    aciertos = aciertos_por_tipo[tipo]
    total_tipo = total_por_tipo.get(tipo, 0)
    if total_tipo > 0:
        porcentaje = (aciertos / total_tipo) * 100
        print(f" - {tipo}: {aciertos} aciertos / {total_tipo} casos ({porcentaje:.2f}%)")
    else:
        print(f" - {tipo}: sin casos en el dataset")

# Calcular métricas avanzadas (Precisión, Recall, F1-Score)

if (VP + FP) > 0:
    precision = VP / (VP + FP)
else:
    precision = 0

if (VP + FN) > 0:
    recall = VP / (VP + FN)
else:
    recall = 0

if (precision + recall) > 0:
    f1_score = 2 * (precision * recall) / (precision + recall)
else:
    f1_score = 0

print("\n📊 Métricas de Clasificación:")
print(f" - Precisión: {precision:.2f}")
print(f" - Recall (Exhaustividad): {recall:.2f}")
print(f" - F1-Score: {f1_score:.2f}")
